In [10]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
NUM_CLASSES = 6
CLASSES = ["sadnesss", "joy", "love", "anger", "fear"]

In [12]:
# Load all data
train_data = pd.read_csv("data/training.csv")
test_data = pd.read_csv("data/test.csv")
validation_data = pd.read_csv("data/validation.csv")

# Separate X's and y's from each other
FEATURE_COLUMNS = ["text"]
LABEL_COLUMN = "label"

X_train = train_data[FEATURE_COLUMNS]
Y_train = train_data[LABEL_COLUMN]

X_test = test_data[FEATURE_COLUMNS]
Y_test = test_data[LABEL_COLUMN]

X_val = validation_data[FEATURE_COLUMNS]
Y_val = validation_data[LABEL_COLUMN]

# These are used to run cross validation
X_train_val = pd.concat([X_train, X_val]) 
Y_train_val = pd.concat([Y_train, Y_test])

# These are used to run val and test for Neural Nets
X_val_test = pd.concat([X_val, X_test])
Y_val_test = pd.concat([Y_val, Y_test])

In [13]:
X_train_maxtoks = X_train['text'].str.len().max()
X_test_maxtoks = X_test['text'].str.len().max()
X_val_maxtoks = X_val['text'].str.len().max() 

max_toks = max(X_train_maxtoks, X_test_maxtoks, X_val_maxtoks)
print(max_toks)

300


# Data Loaders

In [56]:
# Pytorch specific constants
from torch.utils.data import TensorDataset, DataLoader
from extractors.chartok import  CharTokenDataset
from utils.neuralnet import NeuralNetwork
from utils.rnn import LSTMNetwork
from utils.transformer import TransformerEncoder
from utils.trainer import training_loop, evaluate
import torch

BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 500

# Standard MLP

In [18]:
# Because an MLP operates on fixed size inputs, we will use the entire fixed size input for this
train_data = CharTokenDataset(X_train, Y_train, tokenizer=None, max_seq_length=max_toks)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

val_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

In [24]:
mlp = NeuralNetwork(max_toks, [100, 100, 100], NUM_CLASSES, device="cuda")
training_loop(mlp, train_loader, val_loader, epochs=EPOCHS, learning_rate=LEARNING_RATE)
torch.save(mlp.state_dict(), "models/mlpseq")

Epoch 1
train_loss = 1.8439, val_loss = 1.5968
Epoch 2
train_loss = 1.7790, val_loss = 1.5937
Epoch 3
train_loss = 1.2957, val_loss = 1.6226
Epoch 4
train_loss = 1.7348, val_loss = 1.6105
Epoch 5
train_loss = 0.9663, val_loss = 1.6131
Epoch 6
train_loss = 0.9028, val_loss = 1.6133
Epoch 7
train_loss = 0.9092, val_loss = 1.6008
Epoch 8
train_loss = 0.8118, val_loss = 1.6269
Epoch 9
train_loss = 1.1022, val_loss = 1.5985
Epoch 10
train_loss = 1.0939, val_loss = 1.6123
Epoch 11
train_loss = 2.0702, val_loss = 1.5979
Epoch 12
train_loss = 1.2437, val_loss = 1.6203
Epoch 13


KeyboardInterrupt: 

In [ ]:
evaluate(model=mlp, val_dl=val_loader)

loss = 1.6014
accuracy = 0.3417
f1 = 0.2614


(tensor(1.6014), 0.34175, 0.261355887646227)

# RNN

In [64]:
train_data = CharTokenDataset(X_train, Y_train, tokenizer=None, max_seq_length=max_toks, dtype=torch.int32)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

val_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks, dtype = torch.int32)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)


In [65]:
for dat in train_loader:
    x, y= dat
    print(x.shape)
    break

torch.Size([32, 300])


In [67]:
lstm = LSTMNetwork(32,  20, 10, NUM_CLASSES)
training_loop(lstm, train_loader, val_loader, epochs=EPOCHS, learning_rate=LEARNING_RATE)
torch.save(lstm.state_dict(), "models/lstmseq")

Epoch 1
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torch.Size([32, 300, 32])
torc

KeyboardInterrupt: 

# Transformer

In [40]:
train_data = CharTokenDataset(X_train, Y_train, tokenizer=None, max_seq_length=max_toks, dtype=torch.int32)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

val_data = CharTokenDataset(X_val_test, Y_val_test, tokenizer=None, max_seq_length=max_toks, dtype = torch.int32)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

transformer = TransformerEncoder(30, NUM_CLASSES, 40, 5, 4, 0.1)

In [42]:
training_loop(transformer, train_loader, val_loader, epochs=EPOCHS, learning_rate=LEARNING_RATE)
torch.save(transformer.state_dict(), "models/xformerseq")

Epoch 1


KeyboardInterrupt: 